# Deepface Face Verification test 

- Import relevant libraries:

In [1]:
# !pip install deepface
from deepface import DeepFace
from deepface_main_test_func import deepface_benchmark_lfw_split

import matplotlib.pyplot as plt
import pandas as pd                 
import regex as re
import os

# Read the CSV results files saved:
result_df = pd.read_csv('./result/result_deepface.csv')
result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

- Installation and Import checking:  

In [2]:
import importlib.util

packages = ["pandas", "deepface", "regex", "matplotlib", "os"]
for package_name in packages:
    spec = importlib.util.find_spec(package_name)
    if spec is None:
        print(f"Package {package_name} not found!")
    else:
        print(f"Successfully installed and imported {package_name}")

## Manual test:

Example parameter selection: 

In [3]:
# 1. `dataset_choice`: "LFW_gender", "LFW_gender_makeup_heavy", "LFW_gender_makeup_light", "LFW_gender_moustache", "LFW_gender_glasses"
# 2. `model_choice`: "Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace", "ArcFace"
# 3. `image_choices` (Male): "Luis_Ernesto_Derbez_Bautista_0001.jpg", "Junichiro_Koizumi", "Vicente_Fox"
# 4. `image_choices` (Female): "Angelina_Jolie", "Mariangel_Ruiz_Torrealba", "Venus_Williams" 
# 5. `metric_choice`: "cosine", "euclidean", "euclidean_l2"
# 6. `backend_choice`: "opencv", "ssd", "dlib", "mtcnn", "retinaface", "mediapipe"

- Paramater assignment: 

In [28]:
# Parameter for manual test: 
model_choice= "ArcFace"
metric_choice= "euclidean_l2"
backend_choice= "opencv"
target_dataset= "LFW_gender"
target_image= "Angelina_Jolie" 

test_dataset= "LFW_gender"
test_image= "Angelina_Jolie"

# Auto assigned parameters:
DATASET = "LFW_gender"
DATASET_LFW_SPLIT = "LFW_gender"
# male_image_names = os.listdir("./data/LFW_gender/Male/")
male_image_names = os.listdir(f"./data/deepface_{DATASET}/{DATASET_LFW_SPLIT}/Male")
target_gender_choice = "Male" if target_image in male_image_names else "Female"
test_gender_choice = "Male" if test_image in male_image_names else "Female"

target_data_dir_manual = f"./data/{target_dataset}/{target_gender_choice}/{target_image}/{target_image}_0001.jpg"
test_data_dir_manual = f"./data/{test_dataset}/{test_gender_choice}/{test_image}/{test_image}_0002.jpg"

### Manual Face recognition test: 
- Outputs: `Data images`, `model` and Face Recognition `Prediction` [True/False]  

In [29]:
verification = DeepFace.verify(img1_path = target_data_dir_manual, 
                                img2_path = test_data_dir_manual, 
                                model_name=model_choice, 
                                enforce_detection=False,
                                distance_metric= metric_choice, 
                                detector_backend = backend_choice)

print(f"Face recognition test between \n{target_dataset}: [{target_image}] \n{test_dataset}: [{test_image}]\
\nUsing model: {model_choice}  is [{verification['verified']}]")

1/1 [==============================] - 0s 329ms/step
Face recognition test between 
LFW_gender: [Angelina_Jolie] 
LFW_gender: [Angelina_Jolie]
Using model: ArcFace  is [True]


## Parameter settings:
- Bulk Face Recognition test parameter assignment: 

In [3]:
# select a DATASET from data folder:
# LFW, LFW-original, LFW_gender
DATASET = "LFW_gender"

# LFW_gender: Gender split LFW dataset
# LFW_gender_makeup_heavy: Gender split LFW dataset with Heavy makeup
# LFW_gender_makeup_light: Gender split LFW dataset with Light makeup
# LFW_gender_moustache: Gender split LFW dataset with moustache 
# LFW_gender_glasses: Gender split LFW dataset with glasses 
DATASET_LFW_SPLIT = "LFW_gender_makeup_heavy"

# select a model to run:
# models = ["Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace", "ArcFace"]
MODEL = "Facenet512"

# Faces are represented as vectors, a face pair of same person should be similar, calculated by different metrics
# metrics = ["cosine", "euclidean", "euclidean_l2"]
METRIC = "euclidean_l2"

# Face detector, backend that helps aligning the face for better accuracy
# backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
BACKEND = 'opencv'

# Select 'Male' or 'Female'
TEST_GENDER = 'Male'

# Test additional data directory:
# dir_path = f"./additional-data/{DATASET_TEST}"

# Test dataset directory:
dir_path = f"./data/deepface_{DATASET}/{DATASET_LFW_SPLIT}"

- CSV file configuration

In [4]:
# Read CSV files required to index the Dataset images 
# Benchmark Datasets
# benchmark_df = pd.read_csv(f'./data/deepface_{DATASET}_gender_csv/pairs.csv')
benchmark_deepface_df = pd.read_csv(f'./data/deepface_{DATASET}_csv/pairs_{DATASET}.csv')

In [5]:
# models = ["Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace", "ArcFace"]
# gender_lst = ["Male","Female"]
# models = ["Facenet512", "Facenet", "VGG-Face", "OpenFace", "DeepFace"]
# gender_lst = ["Male","Female"]
models = ["Facenet512"]
# gender_lst = ["Male"]
gender_lst = ["Female"]

for mod in models:
    MODEL = mod
    for gn in gender_lst:
        TEST_GENDER = gn

        # face recognition test
        deepface_benchmark_res, perturbed_data = deepface_benchmark_lfw_split(DATASET_LFW_SPLIT, dir_path, benchmark_deepface_df,\
                                                                     MODEL, METRIC, BACKEND, TEST_GENDER)

        # merge results with previous dataframe
        result_df_new = pd.DataFrame([deepface_benchmark_res])
        frames = [result_benchmark_gender_nonperturbed, result_df_new]
        result_benchmark_df = pd.concat(frames)

        result_benchmark_df = result_benchmark_df.reset_index()
        result_benchmark_df.drop(columns=['index'], inplace=True)

        # Output results to CSV and read to current notebook
        result_benchmark_df.to_csv("./result/deepface_benchmark_gender_perturbed.csv", index=False)
        result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')

        # result_benchmark_df.to_csv("./result/deepface_benchmark_gender_nonperturbed.csv", index=False)
        # result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')

1/1 [==============================] - 0s 243ms/step


KeyboardInterrupt: 

In [8]:
result_benchmark_gender_nonperturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender,92.0,97.0,85.0,1000,Male,426,489,11,74,0
1,Facenet512,LFW_gender,91.0,100.0,83.0,1000,Female,414,500,0,86,0
2,Facenet,LFW_gender,73.0,100.0,46.0,1000,Male,232,500,0,268,0
3,Facenet,LFW_gender,73.0,100.0,46.0,1000,Female,232,500,0,268,0
4,VGG-Face,LFW_gender,73.0,68.0,86.0,1000,Male,429,300,200,71,0
5,VGG-Face,LFW_gender,86.0,96.0,75.0,1000,Female,374,485,15,126,0
6,OpenFace,LFW_gender,53.0,94.0,6.0,1000,Male,31,498,2,469,0
7,OpenFace,LFW_gender,58.0,100.0,17.0,1000,Female,83,500,0,417,0
8,DeepFace,LFW_gender,56.0,65.0,23.0,1000,Male,117,438,62,383,0
9,DeepFace,LFW_gender,58.0,84.0,20.0,1000,Female,102,480,20,398,0


## Deepface test: 
- Original benchmark LFW data 

In [7]:
# benchmark testing
# benchmark_df          # 6000 images
# benchmark_deepface_df # 2000 images
# deepface_benchmark_res = deepface_benchmark(DATASET, dir_path, benchmark_deepface_df, MODEL, METRIC, BACKEND)

- Benchmark testing with LFW gender split

In [102]:
# benchmark testing LFW with gender split 
# benchmark_balance         # 3000 images

# currrently using:  
# benchmark_deepface_df     # 2000 images
deepface_benchmark_res, perturbed_data = deepface_benchmark_lfw_split(DATASET_LFW_SPLIT, dir_path, benchmark_deepface_df,\
                                                                     MODEL, METRIC, BACKEND, TEST_GENDER)

1/1 [==============================] - 14s 14s/step


## Result of the Deepface test:

- Benchmark testing:

In [103]:
# Obtain the results for confusion matrix accuracy from the test result dictionary:
cm_acc = deepface_benchmark_res["CM_ACC"]
cm_pre = deepface_benchmark_res["Precision"]
cm_rec = deepface_benchmark_res["Recall"]
truth_pos = deepface_benchmark_res["TP"]
truth_neg = deepface_benchmark_res["TN"]
false_pos = deepface_benchmark_res["FP"]
false_neg = deepface_benchmark_res["FN"]
undetected = deepface_benchmark_res["Undetected"]

# total images
total_imgs = deepface_benchmark_res["Total Images"]

# Format the output for printing
print(f"\
Accuracy of {MODEL} is {cm_acc}% out of {total_imgs} images from {DATASET} dataset.\n\
The Precision is {cm_pre} and Recall is {cm_rec}\n\
===================\n\
True Positive: {truth_pos} \n\
True Negative: {truth_neg} \n\
False Positive: {false_pos} \n\
False Negative: {false_neg} \n\
===================\n\
Undetected: {undetected}")

Accuracy of Facenet is 71.0% out of 1000 images from YTFace_Gender dataset.
The Precision is 85.0 and Recall is 50.0
True Positive: 251 
True Negative: 455 
False Positive: 45 
False Negative: 249 
Undetected: 0


# Saving result:

In [104]:
result_df_new = pd.DataFrame([deepface_benchmark_res])

# result_df
# result_benchmark_gender_nonperturbed
# result_benchmark_gender_perturbed

# frames = [result_benchmark_gender_nonperturbed, result_df_new]
frames = [result_benchmark_gender_perturbed, result_df_new]
result_benchmark_df = pd.concat(frames)

result_benchmark_df = result_benchmark_df.reset_index()
result_benchmark_df.drop(columns=['index'], inplace=True)
result_benchmark_df

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0


- Write to CSV for Benchmark <b>original</b> LFW dataset 

In [ ]:
result_benchmark_df.to_csv("./result/result_deepface.csv", index=False)
result_df = pd.read_csv('./result/result_deepface.csv')

- Write to CSV for <b>non-perturbed data</b> Benchmark LFW gender split

In [84]:
# Write results to csv file
# result_benchmark_df.to_csv("./result/deepface_benchmark_gender.csv", index=False)
# result_benchmark_gender = pd.read_csv('./result/deepface_benchmark_gender.csv')

result_benchmark_df.to_csv("./result/deepface_benchmark_gender_nonperturbed.csv", index=False)
result_benchmark_gender_nonperturbed = pd.read_csv('./result/deepface_benchmark_gender_nonperturbed.csv')
result_benchmark_gender_nonperturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender,92.0,97.0,85.0,1000,Male,426,489,11,74,0
1,Facenet512,LFW_gender,91.0,100.0,83.0,1000,Female,414,500,0,86,0
2,Facenet,LFW_gender,73.0,100.0,46.0,1000,Male,232,500,0,268,0
3,Facenet,LFW_gender,73.0,100.0,46.0,1000,Female,232,500,0,268,0
4,VGG-Face,LFW_gender,73.0,68.0,86.0,1000,Male,429,300,200,71,0
5,VGG-Face,LFW_gender,86.0,96.0,75.0,1000,Female,374,485,15,126,0
6,OpenFace,LFW_gender,53.0,94.0,6.0,1000,Male,31,498,2,469,0
7,OpenFace,LFW_gender,58.0,100.0,17.0,1000,Female,83,500,0,417,0
8,DeepFace,LFW_gender,56.0,65.0,23.0,1000,Male,117,438,62,383,0
9,DeepFace,LFW_gender,58.0,84.0,20.0,1000,Female,102,480,20,398,0


- Write to CSV for <b>perturbed data</b> Benchmark LFW gender split

In [105]:
# Write results to csv file
result_benchmark_df.to_csv("./result/deepface_benchmark_gender_perturbed.csv", index=False)
result_benchmark_gender_perturbed = pd.read_csv('./result/deepface_benchmark_gender_perturbed.csv')
result_benchmark_gender_perturbed

,Model,Dataset,CM_ACC,Precision,Recall,Total Images,Gender,TP,TN,FP,FN,Undetected
0,Facenet512,LFW_gender_makeup_heavy,87.0,94.0,80.0,1000,Male,398,476,24,102,0
1,Facenet512,LFW_gender_makeup_heavy,86.0,99.0,73.0,1000,Female,366,498,2,134,0
2,Facenet,LFW_gender_makeup_heavy,67.0,100.0,35.0,1000,Male,174,500,0,326,0
3,Facenet,LFW_gender_makeup_heavy,69.0,100.0,38.0,1000,Female,192,500,0,308,0
4,VGG-Face,LFW_gender_makeup_heavy,73.0,70.0,81.0,1000,Male,404,327,173,96,0
5,VGG-Face,LFW_gender_makeup_heavy,83.0,96.0,69.0,1000,Female,346,487,13,154,0
6,OpenFace,LFW_gender_makeup_heavy,54.0,95.0,8.0,1000,Male,38,498,2,462,0
7,OpenFace,LFW_gender_makeup_heavy,56.0,100.0,12.0,1000,Female,60,500,0,440,0
8,DeepFace,LFW_gender_makeup_heavy,55.0,62.0,25.0,1000,Male,127,422,78,373,0
9,DeepFace,LFW_gender_makeup_heavy,58.0,79.0,22.0,1000,Female,110,471,29,390,0
